In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.utils as utils
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision
import matplotlib.pyplot as plt
import matplotlib.image as mimg
import os
from tqdm import tqdm

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)


In [ ]:
class dataset(Dataset):
    def __init__(self):
        self.t = transforms.Compose([transforms.ToTensor(),transforms.CenterCrop((350,350)),transforms.Resize((150,150))])
        self.data = os.listdir(img_dir)
        self.trainp = 0.9
    def __len__(self):
        return int(len(self.data)*0.9)
    def __getitem__(self,i):
        img = self.t(mimg.imread(img_dir+self.data[i]))
        return img
d = dataset()
#t = transforms.Compose([transforms.ToTensor(),transforms.CenterCrop((250,250)),transforms.Resize((150,150))])
t = transforms.Compose([transforms.ToTensor(),transforms.Resize((150,150))])
#d = torchvision.datasets.CIFAR10(root= "/", train = True, transform= t, download= True)
loader = DataLoader(d,20)
testa = next(iter(loader))
testimg = testa[4]
plt.imshow(torch.permute(testimg,(1,2,0)))


In [ ]:
IH = 32
IW = 32
c = 8
#^

In [ ]:
"""nn.Conv2d(in_channels= 3 , out_channels= 6 , kernel_size= 5 , stride= 2 , padding= 0 )
nn.Conv2d(in_channels= 6 , out_channels= 9 , kernel_size= 3 , stride= 2 , padding= 0 )
nn.Conv2d(in_channels= 9 , out_channels= 12 , kernel_size= 3 , stride= 2 , padding= 0 )
nn.Conv2d(in_channels= 12 , out_channels= 15 , kernel_size= 3 , stride= 1 , padding= 0 )
nn.ConvTranspose2d(in_channels= 15 , out_channels= 12 , kernel_size= 3 , stride= 1 , padding= 0 )
nn.ConvTranspose2d(in_channels= 12 , out_channels= 9 , kernel_size= 3 , stride= 2 , padding= 0 )
nn.ConvTranspose2d(in_channels= 9 , out_channels= 6 , kernel_size= 3 , stride= 2 , padding= 0 )
nn.ConvTranspose2d(in_channels= 6 , out_channels= 3 , kernel_size= 5 , stride= 2 , padding= 0 )
nn.ConvTranspose2d(in_channels= 3 , out_channels= 3 , kernel_size= 3 , stride= 1 , padding= 0 )
nn.ConvTranspose2d(in_channels= 3 , out_channels= 3 , kernel_size= 4 , stride= 1 , padding= 0 )"""

In [ ]:
class nblock(nn.Module):
    def __init__(self,c,encoder:bool,k, s,an ):
        self.helper = [c[0]]
        super(nblock, self).__init__()
        self.enc = encoder
        self.an = an
        self.c = c
        self.k = k
        self.s = s
        layers = []
        skips = []
        for i in range(0,len(k)):
            l4 = self.makel(i)
            layers.append(l4[0])
            if((i+1)%an==0):
                skips.append(l4[1])
        self.layers = nn.ModuleList(layers)
        self.skips = nn.ModuleList(skips)
    def makel(self,i):
        if (i+1)%self.an==0:
            rv =  (nn.Sequential(nn.ConvTranspose2d(in_channels=self.c[i],out_channels= self.c[i+1] ,kernel_size= self.k[i],stride= self.s[i]),nn.LeakyReLU()),nn.Conv2d(self.helper[-1],self.c[i+1],1,1),) if self.enc ==False else (nn.Sequential(nn.Conv2d(in_channels=self.c[i],out_channels= self.c[i+1] ,kernel_size= self.k[i],stride= self.s[i]),nn.LeakyReLU()),nn.Conv2d(self.helper[-1],self.c[i+1],1,1))
            self.helper.append(self.c[i+1])
        else:
            return(nn.Sequential(nn.ConvTranspose2d(in_channels=self.c[i],out_channels= self.c[i+1] ,kernel_size= self.k[i],stride= self.s[i]))) if self.enc == False else nn.Sequential(nn.Conv2d(in_channels=self.c[i],out_channels= self.c[i+1] ,kernel_size= self.k[i],stride= self.s[i]),)
        return rv

    def forward(self,x):
        x0=[x.clone()]
        y=x
#        for i in range(len(self.layers)):
#            y = self.layers[i][0](y)
#            if((i+1))%self.an==0:
#                y = torch.add(y,self.layers[i][1](transforms.Resize(y.shape[-2:])(x0)))
#                x0 = y
        n=0
        for i, l in enumerate(self.layers):
            y = l(y)
            if(i+1)%self.an==0:
                #y = torch.add(y,self.skips[n](transforms.Resize(y.shape[-2:])(x0[-1])))
                #x0.append(y.clone())
                n+=1
        return y
        

In [ ]:
class rblock(nn.Module):
    def __init__(self,c,encoder:bool,k, s,an ):
        self.helper = [c[0]]
        super(rblock, self).__init__()
        self.enc = encoder
        self.an = an
        self.c = c
        self.k = k
        self.s = s
        layers = []
        skips = []
        for i in range(0,len(k)):
            l4 = self.makel(i)
            layers.append(l4[0])
            if((i+1)%an==0):
                skips.append(l4[1])
        self.layers = nn.ModuleList(layers)
        self.skips = nn.ModuleList(skips)
    def makel(self,i):
        if (i+1)%self.an==0:
            rv =  (nn.Sequential(nn.ConvTranspose2d(in_channels=self.c[i],out_channels= self.c[i+1] ,kernel_size= self.k[i],stride= self.s[i]),nn.LeakyReLU()),nn.Conv2d(self.helper[-1],self.c[i+1],1,1)) if self.enc ==False else (nn.Sequential(nn.Conv2d(in_channels=self.c[i],out_channels= self.c[i+1] ,kernel_size= self.k[i],stride= self.s[i]),nn.LeakyReLU()),nn.Conv2d(self.helper[-1],self.c[i+1],1,1))
            self.helper.append(self.c[i+1])
        else:
            return(nn.Sequential(nn.ConvTranspose2d(in_channels=self.c[i],out_channels= self.c[i+1] ,kernel_size= self.k[i],stride= self.s[i]))) if self.enc == False else nn.Sequential(nn.Conv2d(in_channels=self.c[i],out_channels= self.c[i+1] ,kernel_size= self.k[i],stride= self.s[i]))
        return rv

    def forward(self,x):
        x0=[x.clone()]
        y=x
#        for i in range(len(self.layers)):
#            y = self.layers[i][0](y)
#            if((i+1))%self.an==0:
#                y = torch.add(y,self.layers[i][1](transforms.Resize(y.shape[-2:])(x0)))
#                x0 = y
        n=0
        for i, l in enumerate(self.layers):
            y = l(y)
            if(i+1)%self.an==0:
                y = torch.add(y,self.skips[n](transforms.Resize(y.shape[-2:])(x0[-1])))
                x0.append(y.clone())
                n+=1
        return y
        

In [ ]:

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.net = nblock((6,3,3,2,2,1,1),True,(3,3,3,3,3,3),(2,2,1,1,1,1),20)
        self.lin = nn.Sequential(nn.Linear(784,500),nn.Linear(500,100),nn.Linear(100,1))
    def forward(self, x):
        return nn.Sigmoid()(self.lin(nn.Flatten()(self.net(x))))

Disc = Discriminator().to(device)
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.net2 = nblock((3,9,12,15,15),True,(3,3,3,3),(2,2,2,2),2)
        self.net = rblock((15,15,12,9,3,3),False,(3,3,3,3,8),(2,2,2,2,1),2)
    def forward(self, x):
        x2 = self.net2(x)
        return self.net(x2)
Gen = Decoder().to(device)



In [ ]:
Disc

In [ ]:
Gen.train()
Disc.train()

lr = 0.001
gOpt = torch.optim.Adam(Gen.parameters(), lr = 0.0002)
dOpt = torch.optim.Adam(Disc.parameters(), lr =0.0002)
mae = nn.L1Loss()
bce = nn.BCELoss()

In [ ]:
torch.autograd.set_detect_anomaly(True)

In [ ]:
for epoch in range(0,10):
    for l in enumerate(tqdm(loader)):
        Disc.zero_grad()
        im = l[1].to(device)
        recon = Gen(im)
        doutr = Disc(torch.cat((im,im),1))
        dlossr = bce(doutr,torch.ones_like(doutr))
        doutf = Disc(torch.cat((recon,im),1))
        dlossf = bce(doutf,torch.zeros_like(doutf))
        dlossr.backward(retain_graph = True)
        dlossf.backward(retain_graph = True)
        dOpt.step()
        
        Gen.zero_grad()
        recon2= Gen(im)
        doutf2 = Disc(torch.cat((recon2,im),1))
        gloss = bce(doutf2,torch.ones_like(doutf2))+(4*mae(recon,im))
        gloss.backward(retain_graph = True)
        gOpt.step()

        if(l[0]%100==0):
            f, axarr = plt.subplots(1,2) 
            axarr[0].imshow(torch.permute(Gen(testa.to(device)).to("cpu").detach()[4],(1,2,0)))
            axarr[1].imshow(torch.permute(testa[4],(1,2,0)).detach().to("cpu"))
            plt.show()
            print(dlossr,gloss)
        if(epoch%3==0):
            torch.save(Gen.state_dict(), "gen"+str(epoch))
            torch.save(Disc.state_dict(), "disc"+str(epoch))
        torch.cuda.empty_cache()


        

        recon3 = Gen(im)
        doutf3 = Disc(torch.cat((recon3,im),1))
        eloss = bce(doutf3,torch.zeros_like(doutf3)) +(4*mae(im,recon3))
        del im
        del recon
        del doutf
        del doutr
        eloss.backward()


In [ ]:
for epoch in range(0,10):
    for l in enumerate(tqdm(loader)):
        Disc.zero_grad()
        im = l[1].to(device)
        recon = Gen(im)
        #print(recon.shape,im.shape)
        doutr = Disc(torch.cat((im,im),1))
        dlossr = bce(doutr,torch.ones_like(doutr))
        doutf = Disc(torch.cat((recon,im),1))
        dlossf = bce(doutf,torch.zeros_like(doutf))
        dlossr.backward(retain_graph = True)
        dlossf.backward(retain_graph = True)
        dOpt.step()
        
        Gen.zero_grad()
        recon2= Gen(im)
        doutf2 = Disc(torch.cat((recon2,im),1))
        gloss = bce(doutf2,torch.ones_like(doutf2))+(4*mae(recon,im))
        gloss.backward(retain_graph = True)
        gOpt.step()

        if(l[0]%1000==0):
            f, axarr = plt.subplots(1,2) 
            axarr[0].imshow(torch.permute(Gen(testa.to(device)).to("cpu").detach()[4],(1,2,0)))
            axarr[1].imshow(torch.permute(testa[4],(1,2,0)).detach().to("cpu"))
            plt.show()
            print(dlossr,gloss)
        if(epoch%3==0):
            torch.save(Gen.state_dict(), "gen"+str(epoch))
            torch.save(Disc.state_dict(), "disc"+str(epoch))
        torch.cuda.empty_cache()


        

        recon3 = Gen(im)
        doutf3 = Disc(torch.cat((recon3,im),1))
        eloss = bce(doutf3,torch.zeros_like(doutf3)) +(4*mae(im,recon3))
        del im
        del recon
        del doutf
        del doutr
        eloss.backward()

In [ ]:
torch.save(Gen.state_dict(), "Gen.model")
torch.save(Disc.state_dict(), "Disc.model")

In [ ]:
class dataset2(Dataset):
    def __init__(self):
        self.t = transforms.Compose([transforms.ToTensor(),transforms.CenterCrop((150,150)),transforms.Resize((150,150))])
        self.trainp = 0.9
    def __len__(self):
        return 20
    def __getitem__(self,i):
        img = self.t(mimg.imread("/input/testimage/rock.jpg"))
        return img
d = dataset2()
#t = transforms.Compose([transforms.ToTensor(),transforms.CenterCrop((250,250)),transforms.Resize((150,150))])
loader = DataLoader(d,20)
testa = next(iter(loader))
testimg = testa[0]
testimg = torch.permute(testimg,(1,2,0))
plt.imshow(testimg)


In [ ]:
reconstructed = torch.permute(Gen(torch.unsqueeze(testa[0].to(device),0)).detach().to("cpu"),(0,2,3,1))[0]
plt.imshow(reconstructed)